# MNIST Classifier with data augmentation

**Developed by**: Jhonnatan Torres

**Goal**: Build a MNIST classifier with data augmentation to be used in the following Streamlit app **[MNIST Classifier](https://share.streamlit.io/jotor/streamlitmnist/main/mnist.py)**

In [1]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [2]:
(training_images, training_labels), (test_images, test_labels) = mnist

In [3]:
test_images.shape[0]

10000

In [4]:
training_images = training_images.reshape(training_images.shape[0], 28, 28, 1)

In [5]:
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
dg = ImageDataGenerator(rotation_range=20, width_shift_range=0.4, height_shift_range=0.4, rescale= 1/ 255, fill_mode='constant')
tdg = ImageDataGenerator(rescale= 1/ 255)

In [8]:
it = dg.flow(training_images, training_labels)
tit = tdg.flow(test_images, test_labels)

In [9]:
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
tf.keras.layers.Conv2D(32, (2,2), activation='relu', input_shape=(28, 28, 1)),
tf.keras.layers.MaxPooling2D((2,2)),
tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
tf.keras.layers.MaxPooling2D((2,2)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(64, activation='relu'),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(10, activation='softmax')
])

In [10]:
from tensorflow.keras.optimizers import SGD, Adam
#model.compile(optimizer=SGD(learning_rate=0.1, momentum=0.9), loss='sparse_categorical_crossentropy', metrics='accuracy')
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics='accuracy')

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 32)        4128      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0

In [12]:
model.fit(it, epochs=10, validation_data=tit, batch_size=32, steps_per_epoch=len(it))

Epoch 1/10
1875/1875 [==============================] - 64s 18ms/step - loss: 1.3161 - accuracy: 0.5292 - val_loss: 0.2167 - val_accuracy: 0.9434
Epoch 2/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.6874 - accuracy: 0.7778 - val_loss: 0.1343 - val_accuracy: 0.9593
Epoch 3/10
1875/1875 [==============================] - 35s 18ms/step - loss: 0.5171 - accuracy: 0.8381 - val_loss: 0.0754 - val_accuracy: 0.9766
Epoch 4/10
1875/1875 [==============================] - 35s 18ms/step - loss: 0.4262 - accuracy: 0.8680 - val_loss: 0.0672 - val_accuracy: 0.9798
Epoch 5/10
1875/1875 [==============================] - 35s 18ms/step - loss: 0.3774 - accuracy: 0.8837 - val_loss: 0.0530 - val_accuracy: 0.9826
Epoch 6/10
1875/1875 [==============================] - 35s 18ms/step - loss: 0.3497 - accuracy: 0.8921 - val_loss: 0.0526 - val_accuracy: 0.9847
Epoch 7/10
1875/1875 [==============================] - 35s 18ms/step - loss: 0.3243 - accuracy: 0.9032 - val_loss: 0.0428 -

In [13]:
model.save('mnist.h5')

In [14]:
%cd /content/
%ls

/content
mnist.h5  sample_data/


In [15]:
model.history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [16]:
import numpy as np
preds = np.argmax(model.predict(test_images / 255 ), axis=1)

In [17]:
from sklearn.metrics import confusion_matrix

In [18]:
print(confusion_matrix(test_labels, preds))

[[ 971    0    2    0    0    1    5    0    0    1]
 [   0 1131    1    0    0    1    0    2    0    0]
 [   2    2 1012    6    0    0    0   10    0    0]
 [   0    0    0 1004    0    4    0    1    1    0]
 [   0    1    0    0  976    0    1    0    0    4]
 [   1    0    0    4    0  882    4    0    1    0]
 [   2    5    0    0    0    2  947    0    2    0]
 [   0    7    6    0    0    0    0 1015    0    0]
 [   0    0    2    2    0    1    1    0  965    3]
 [   7    0    0    0   11    6    0    4    4  977]]
